# Looking at merged tract-patch catalogs in Run 1.1 leading up to DC2
Michael Wood-Vasey

In [ ]:
import os

import numpy as np

%matplotlib inline 
import matplotlib.pyplot as plt

import pandas as pd

In [ ]:
# Some plotting choices
plt.rcParams['figure.figsize'] = (8, 8)

# cmap = 'Oranges'
cmap = 'viridis_r'

In [ ]:
merged_tract_data_dir = '/global/projecta/projectdirs/lsst/global/in2p3/Run1.1-test2/summary/'

In [ ]:
tract, patch = 4849, 16
basename = 'merged_tract_%d.hdf5' % tract
key = 'coadd_%d_%2d' % (tract, patch)

In [ ]:
merged_tract_file = os.path.join(merged_tract_data_dir, basename)

In [ ]:
df = pd.read_hdf(merged_tract_file, key=key)

In [ ]:
print(len(df), "coadd objects")

In [ ]:
plt.figure(figsize=(8,8))
plt.scatter(df['g_mag'] - df['r_mag'], df['r_mag'] - df['i_mag'], marker='.')
plt.xlim(-1, +2)
plt.ylim(-1, +2)
plt.xlabel('g-r')
plt.ylabel('r-i')

That's a bit of a mess, but maybe we're starting to see a bit of some structure resolving.  Let's do the traditional simple 2D histogram version:

Let's overplot a rough stellar locus.  This is just a simple approximation with two lines.  See, e.g., Ivezic+2007, AJ, 134, 973. Fig. 3 (http://iopscience.iop.org/article/10.1086/519976/pdf)

In [ ]:
def simple_stellar_locus_gmr_rmi(color='red', linestyle='--', linewidth=2.5):
    m_stars_gmr = [+1.4, +1.4]
    m_stars_rmi = [+0.5, +1.5]

    other_stars_gmr = [-0.5, +1.4]
    other_stars_rmi = [-0.4, +0.5]

    model_gmr = other_stars_gmr + m_stars_gmr
    model_rmi = other_stars_rmi + m_stars_rmi
    
    return model_gmr, model_rmi

    
def get_stellar_locus_davenport(color1='gmr', color2='rmi',
                                datafile='Davenport_2014_MNRAS_440_3430_table1.txt'):
    data = pd.read_table(datafile, sep='\s+', header=1)
    return data[color1], data[color2]
    
    
def plot_stellar_locus(color1='gmr', color2='rmi',
                       color='red', linestyle='--', linewidth=2.5):
    model_gmr, model_rmi = get_stellar_locus_davenport(color1, color2)
    plot_kwargs = {'linestyle': linestyle, 'linewidth': linewidth, 'color': color,
                   'scalex': False, 'scaley': False}
    plt.plot(model_gmr, model_rmi, **plot_kwargs)

In [ ]:
def plot_color_color(z, color1, color2, range1=(-1, +2), range2=(-1, +2), bins=31):
    """Plot a color-color diagram.  Overlay stellar locus"""
    band1, band2 = color1[0], color1[-1]
    band3, band4 = color2[0], color2[-1]
    H, xedges, yedges = np.histogram2d(
        z['%s_mag' % band1] - z['%s_mag' % band2],
        z['%s_mag' % band3] - z['%s_mag' % band4],
        range=(range1, range2), bins=bins)
        
    zi = H.T
    xi = (xedges[1:] + xedges[:-1])/2
    yi = (yedges[1:] + yedges[:-1])/2

    plt.figure(figsize=(8, 8))
    plt.pcolormesh(xi, yi, zi, cmap=cmap)
    plt.contour(xi, yi, zi)
    plt.xlabel('%s-%s' % (band1, band2))
    plt.ylabel('%s-%s' % (band3, band4))

    plot_stellar_locus(color1, color2)

In [ ]:
plt.figure(figsize=(8,6.5))
plt.hist2d(df['g_mag']-df['r_mag'], df['r_mag']-df['i_mag'],
           cmap=cmap, range=((-1, +2), (-1, +2)), bins=51)
plt.colorbar()
plt.xlabel('g-r')
plt.ylabel('r-i')

plot_stellar_locus('gmr', 'rmi')

Clean this up a bit to only include stars with SNR > 3 in g and r

In [ ]:
snr_threshold = 25
mag_err_threshold = 1/snr_threshold
good_snr = df[(df['g_mag_err'] < mag_err_threshold) & (df['r_mag_err'] < mag_err_threshold)]

In [ ]:
bright_snr_threshold = 100
mag_err_threshold = 1/bright_snr_threshold
bright_stars = df[(df['g_mag_err'] < mag_err_threshold) & (df['r_mag_err'] < mag_err_threshold)]

In [ ]:
safe_max_extended = 1.0
stars = good_snr[good_snr['base_ClassificationExtendedness_value'] < safe_max_extended]
galaxies = good_snr[good_snr['base_ClassificationExtendedness_value'] >= safe_max_extended]

In [ ]:
print("%d stars (SNR > %.0f)" % (len(stars), snr_threshold))
print("%d bright stars (SNR > %.0f)" % (len(bright_stars), bright_snr_threshold))

In [ ]:
hist_kwargs = {'bins': np.linspace(-2, 3, 51),
               'range': (-2, +3),
               'linewidth': 4,
               'histtype': 'step'}
          
plt.hist(df['g_mag'] - df['r_mag'], label='all', color='grey', **hist_kwargs)
plt.hist(good_snr['g_mag'] - good_snr['r_mag'], label='SNR > %.0f' % snr_threshold, color='black',
         **hist_kwargs)
plt.hist(galaxies['g_mag'] - galaxies['r_mag'], label='galaxies', **hist_kwargs)
plt.hist(stars['g_mag'] - stars['r_mag'], label='stars', **hist_kwargs)
plt.xlabel('g-r')
plt.ylabel('# / 0.5 mag bin')
plt.legend()

In [ ]:
plot_color_color(galaxies, 'gmr', 'rmi')

In [ ]:
plot_color_color(stars, 'gmr', 'rmi')

In [ ]:
plt.scatter(df['r_mag'], 1/df['r_mag_err'], label='1/r_mag_err')
plt.xlabel('r')
plt.ylabel('1/r_err')
plt.axhline(bright_snr_threshold, linestyle='--', color='red', label='Bright star')
plt.axhline(snr_threshold, linestyle='--', color='orange', label='Good SNR')
plt.legend()
plt.xlim(15, 28)
# plt.ylim(0, 1.0)

In [ ]:
plot_color_color(stars, 'rmi', 'imz')

See, e.g., http://www.astroml.org/examples/datasets/plot_sdss_imaging.html

In [ ]:
z = galaxies
plt.scatter(z['g_mag'] - z['r_mag'], z['g_mag'], marker='.')
plt.xlim(-1, +2)
plt.ylim(25, 15)

In [ ]:
z = stars
H, xedges, yedges = np.histogram2d(z['g_mag']-z['r_mag'], z['g_mag'],
                                    range=((-1, +2), (15, 25)), bins=31)
zi = H.T
xi = (xedges[1:] + xedges[:-1])/2
yi = (yedges[1:] + yedges[:-1])/2

plt.pcolormesh(xi, yi, zi, cmap=cmap)
plt.contour(xi, yi, zi)
plt.xlabel('g-r')
plt.ylabel('g')
plt.ylim(25, 15)

## Appendix

## Generic Catalog Reader (GCR)

After a little bit you'll find yourself wanting to ask interesting questions, like how well does the DC2 analysis recover the input sources?  Or easily loading in all of the patches of a tract.  A convenient framework for generalizing analyses of DESC-DC related catalogs is the Generic Catalog Reader:

* GCR is a general utility
https://github.com/yymao/generic-catalog-reader
* The catalogs supporting DC2 are in:
https://github.com/LSSTDESC/gcr-catalogs

See 'DC2_Static_Coadd_Catalog_reader.ipynb' notebook by Yao-Yuan Mao

https://github.com/LSSTDESC/DC2_Repo/blob/master/Notebooks/DC2%20Static%20Coadd%20Catalog%20reader.ipynb

We don't have a package management set up for our DESC+DC2 products, so that notebook starts with a bit of explicit insertion of DC2 scripts into the path.

## Density Estimation
We could entertain ourselves by doing a somewhat better density estimation
E.g., a Gaussian kernel density estimation.

In [ ]:
# https://stackoverflow.com/questions/19390320/scatterplot-contours-in-matplotlib
def bin_kde(x, y, nbins=20, **kwargs):
    from scipy.stats import kde
    k = kde.gaussian_kde(x, y)
    xi, yi = np.mgrid[x.min():x.max():nbins*1j, y.min():y.max():nbis*1j]
    zi = k(np.vstack([xi.flatten(), yi.flatten()]))

# This doesn't exactly work, I've done something wrong here.
xi, yi, zi = bin_kde(stars['g_mag']-stars['r_mag'], stars['r_mag']-stars['i_mag'])